In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)
from swift.utils import seed_everything
from swift.tuners import Swift
import json

#微调目录
model_dir1 = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/mistral-7b-chat-v2/v7-20231228-133749/checkpoint-17477'
model_dir2 = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/mistral-7b-chat-v2/v9-20240104-151231/checkpoint-250'

#llama
model_dir3 = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/llama2-7b-chat/v4-20240113-171722/checkpoint-17477'
model_dir4 = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/llama2-7b-chat/v5-20240115-130325/checkpoint-250'



#llama
model_type3 = ModelType.llama2_7b_chat
template_type3 = get_default_template_type(model_type3)
model_llama, tokenizer_llama = get_model_tokenizer(model_type3, model_kwargs={'device_map': 'auto'}, model_dir="/home/css/models/Llama-2-7b-chat-hf")
model_llama.generation_config.max_new_tokens = 4096
model_llama = Swift.from_pretrained(model_llama, model_dir3, inference_mode=True)
model_llama = Swift.from_pretrained(model_llama, model_dir4, inference_mode=True)
template_llama = get_template(template_type3, tokenizer_llama)

In [ ]:
model_llama.generation_config.top_p=0.75
model_llama.generation_config.temperature=0.01
model_llama.generation_config.update(repetition_penalty=2.0,top_k=40,num_beams=4)

content = '''"Ukranian Prime Min Viktor F Yanukovich visits Washington for low key meetings with Vice Pres Dick Cheney and Sec of State Condoleezza Rice; says Ukraine is in no hurry to join NATO or European Union (S)"'''

# content = '''SaharaReporters has learned from reliable sources at the Nigerian Presidency that President Muhammadu Buhari, who has been in the UK since January 19, is receiving \u201cintense treatment\u201d for a renewed flare-up of prostate issues. He had undergone surgical treatment for prostate cancer soon after losing the 2011 presidential election to former President Goodluck Jonathan.\nIn addition to his UK doctors, President Buhari has been under the care of two Nigerian physicians, Dr. Suhayb Sanusi Rafindadi, who is his Chief Personal Physician, and Dr. Ugorji Ogbonna, who had a medical practice for many years in Kano before relocating to the UK. Even though he is from the southeast of Nigeria, Dr. Ogbonna is very close to numerous powerful northern politicians, and one of his sons converted to Islam, according to a source. While Dr. Rafindadi accompanied the president on his medical trips abroad and Dr. Ogbonna often liaise with Mr. Buhari\u2019s physicians in the UK to arrange for his treatments.\nOur sources indicated that Mr. Buhari\u2019s original surgical treatment for prostate cancer had been declared successful. However, when a nagging ear infection forced President Buhari to visit his longtime physician in the UK last year, the prognosis led the president to seek two separate medical opinions, in France and later Germany. Doctors in both countries reportedly told him it was urgent to see his doctors in the UK. Subsequently, Mr. Buhari, who was visiting France for a conference, moved from Paris to London, declaring that he needed a vacation.\nMr. Buhari\u2019s UK doctors advised that he needed to stay put in London for another surgery, but the president bowed to pressure from members of his inner circle and decided to make a premature return to Abuja out of political expediency. However, before President Buhari left London, his doctors there removed polyps from his nostrils in a surgical procedure to ease his breathing.\nIn 2016, President Buhari made trips to the UK in February and June to consult his UK doctors on a variety of health issues, including his ear infection, said our sources.\nSaharaReporters learned that President Buhari currently receives intense treatment for a prostate-related ailment. The treatment at a point severely affected his voice and appetite. He has progressively lost weight and has had to be force-fed on occasion on the orders of his doctors. One source said Mr. Buhari\u2019s treatments had been compounded by his age, which his UK doctors believe to be more than 80 rather than his official \u201cage\u201d of 74 years old.\nInformation gleaned from our sources indicated an attempt to obfuscate the precise nature of Mr. Buhari\u2019s illness, signaling efforts by competing factions around the president to keep the Nigerian public uninformed. According to our sources, four different versions of President Buhari's condition were being circulated. One version, traced to his wife, Aisha Buhari, is that the president has \u201cinternal organ\u201d issues.\nOur sources disclosed that a cabal led by President Buhari's cousin, Mamman Daura, reportedly sent Mrs. Aisha Buhari away from the UK, asking her to make a temporary visit to Saudi Arabia. To justify Aisha Buhari\u2019s exit from London, Mr. Daura reportedly claimed that the president\u2019s condition appeared to worsen each time his wife was around him. He reportedly told Mrs. Buhari that part of her husband's illness was a result of \u201ca spiritual attack,\u201d and asked her to proceed to Saudi Arabia to pray for him. From Saudi Arabia, Mrs. Buhari returned to Nigeria about two weeks ago. One source claimed that she had not returned to London ever since.\nOur sources said Mr. Daura often handled the daily briefing of select Presidency officials about the president's health. According to them, the president\u2019s cousin continues to claim that Mr. Buhari was only exhausted and needed adequate rest before returning to Nigeria ready to take over the mantle of leadership once again.\nAs SaharaReporters revealed last weekend, the doctors treating Mr. Buhari in the UK have told him in clear terms that he ought to shelf any plans to return to serious work and stay back in London for as long as four months to receive a full course of treatment. One source indicated that members of the president\u2019s inner circle were yet to fully embrace the doctors\u2019 recommendation and communicate the information to Nigerians because they fear the loss of political influence.\nOn his part, President Buhari had reportedly told his inner political circle that he was in no hurry to return to Nigeria, adding that he was willing to let the expert advice of his doctors to prevail. However, some of his die-hard associates have not given up plotting to daily to convince him to return to Abuja in defiance of medical advice.\nPresident Buhari reportedly communicates regularly with Acting President Yemi Osinbajo, encouraging him to carry on the task of governing Nigeria while he undertakes necessary treatment needed to keep him alive.'''
eval_prompt = f"### Instruction: According to the following information: <{content}> Answer the question. Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination,year-month],just country name.\n### Response:\n"

# eval_prompt = f"### Instruction: According to the following information, answer the question. Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination,year-month],just country name.\n### Information:{content}\n### Response:\n"

print(eval_prompt)

response_llama, history_llama = inference(
    model_llama, template_llama, eval_prompt,
)

print(response_llama)
